In [ ]:
import os
import cv2

# ✅ Path to your main dataset folder
base_dir = r"/content/drive/MyDrive/Colab Notebooks/archive (7)"
target_size = (224, 224)  # width, height

for class_name in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"📏 Resizing images in: {class_name}")

    for img_name in os.listdir(class_path):
        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            print(f"⚠ Skipped unreadable image: {img_path}")
            continue

        # Resize image
        resized_img = cv2.resize(img, target_size)

        # Overwrite the image file
        cv2.imwrite(img_path, resized_img)

print("✅ All images resized to 224x224 successfully!")

📏 Resizing images in: Pulmonery Inflamatory Process (Pneumonia)
📏 Resizing images in: Mediastrinal Alterations
📏 Resizing images in: Thoracic Alterations
📏 Resizing images in: Normal Anatomy
📏 Resizing images in: Encapsulated Listens
📏 Resizing images in: Obstructive Disease
📏 Resizing images in: Dercreased Density
📏 Resizing images in: Increased Density
📏 Resizing images in: Infectious Degenerative
✅ All images resized to 224x224 successfully!


In [ ]:
import os
import shutil
import random

# Paths
base_dir = "/content/drive/MyDrive/Colab Notebooks/archive (7)"   # main folder that has category subfolders
train_dir = "/content/drive/MyDrive/Colab Notebooks/train"
test_dir = "/content/drive/MyDrive/Colab Notebooks/test"

# Create target folders if they don’t exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get all category folders
categories = os.listdir(base_dir)

# Split ratio
train_size = 400
test_size = 100

for category in categories:
    category_path = os.path.join(base_dir, category)
    if not os.path.isdir(category_path):
        continue

    images = os.listdir(category_path)
    random.shuffle(images)

    # Split images
    train_imgs = images[:train_size]
    test_imgs = images[train_size:train_size + test_size]

    # Create category folders inside train and test
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)

    # Move or copy files
    for img in train_imgs:
        shutil.copy(os.path.join(category_path, img), os.path.join(train_dir, category, img))
    for img in test_imgs:
        shutil.copy(os.path.join(category_path, img), os.path.join(test_dir, category, img))

    print(f"✅ {category}: {len(train_imgs)} train, {len(test_imgs)} test")

print("🎯 Dataset successfully split into train and test folders!")


✅ Pulmonery Inflamatory Process (Pneumonia): 400 train, 100 test
✅ Mediastrinal Alterations: 400 train, 100 test
✅ Thoracic Alterations: 400 train, 100 test
✅ Normal Anatomy: 400 train, 100 test
✅ Encapsulated Listens: 400 train, 100 test
✅ Obstructive Disease: 400 train, 100 test
✅ Dercreased Density: 400 train, 100 test
✅ Increased Density: 400 train, 100 test
✅ Infectious Degenerative: 400 train, 100 test
🎯 Dataset successfully split into train and test folders!


In [ ]:
import os

train_dir = "/content/drive/MyDrive/Colab Notebooks/train"
test_dir = "/content/drive/MyDrive/Colab Notebooks/test"

def check_image_counts(base_dir, expected_count):
    for category in os.listdir(base_dir):
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            count = len(os.listdir(category_path))
            if count != expected_count:
                print(f"⚠️ {category}: {count} images (Expected {expected_count})")
            else:
                print(f"✅ {category}: {count} images OK")

print("Checking train folder:")
check_image_counts(train_dir, 400)

print("\nChecking test folder:")
check_image_counts(test_dir, 100)


Checking train folder:
⚠️ Pulmonery Inflamatory Process (Pneumonia): 498 images (Expected 400)
⚠️ Mediastrinal Alterations: 495 images (Expected 400)
⚠️ Thoracic Alterations: 495 images (Expected 400)
⚠️ Normal Anatomy: 496 images (Expected 400)
⚠️ Encapsulated Listens: 498 images (Expected 400)
⚠️ Obstructive Disease: 500 images (Expected 400)
⚠️ Dercreased Density: 494 images (Expected 400)
⚠️ Increased Density: 497 images (Expected 400)
⚠️ Infectious Degenerative: 498 images (Expected 400)

Checking test folder:
⚠️ Pulmonery Inflamatory Process (Pneumonia): 235 images (Expected 100)
⚠️ Mediastrinal Alterations: 240 images (Expected 100)
⚠️ Thoracic Alterations: 245 images (Expected 100)
⚠️ Normal Anatomy: 244 images (Expected 100)
⚠️ Encapsulated Listens: 245 images (Expected 100)
⚠️ Obstructive Disease: 245 images (Expected 100)
⚠️ Dercreased Density: 247 images (Expected 100)
⚠️ Increased Density: 247 images (Expected 100)
⚠️ Infectious Degenerative: 260 images (Expected 100)


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Paths
train_dir = "/content/drive/MyDrive/Colab Notebooks/train"
test_dir = "/content/drive/MyDrive/Colab Notebooks/test"

# Image size and parameters
img_size = (224, 224)
batch_size = 32

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load train and test sets
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Build CNN model from scratch
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=20
)

# Evaluate model
loss, acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {acc*100:.2f}%")

# Save the model
#model.save("xray_classifier_model.h5")

KeyboardInterrupt: 

In [ ]:
model.save("xray_classifier_model.h5")

In [ ]:
import cv2
import numpy as np

# Read the image
img = cv2.imread("/content/drive/MyDrive/Colab Notebooks/train/Dercreased Density/03 (100).jpeg")

# Check shape
print("Image shape:", img.shape)

In [ ]:
import numpy as np

# Calculate the average pixel value
average_pixel_value = np.mean(img)

print(f"Average pixel value: {average_pixel_value:.2f}")

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')
])

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,863 (42.61 MB)

 Trainable params: 11,169,863 (42.61 MB)

 Non-trainable params: 0 (0.00 B)